In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [2]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')


In [4]:
# Some baselines that were executed in TIRA

bm25_baseline = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)', pt_dataset)
sparse_cross_encoder = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', pt_dataset)
rank_zephyr = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', pt_dataset)

In [9]:
# This assumes we have execited the ../baseline-retrieval-system/baseline-retrieval-system.ipynb notebook before.
bm25_and_discounted_pagerank = pt.io.read_results('../runs/bm25_and_discounted_pagerank_run.txt')
bm25_and_discounted_pagerank_reranked = pt.io.read_results('../runs/bm25_and_discounted_pagerank_reranked_run.txt')

bm25_ReRank = pt.io.read_results('../runs/bm25-ReRank_run.txt')

Doc2QueryT5_BM25 = pt.io.read_results('../runs/Doc2QueryT5-BM25_run.txt')
Doc2QueryT5_BM25_ReRank = pt.io.read_results('../runs/Doc2QueryT5-BM25_ReRank_run.txt')


pt.Experiment(
    [bm25_and_discounted_pagerank, bm25_and_discounted_pagerank_reranked, bm25_ReRank, Doc2QueryT5_BM25, Doc2QueryT5_BM25_ReRank, bm25_baseline, sparse_cross_encoder, rank_zephyr],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100"],
    names=["bm25_and_discounted_pagerank", "bm25_and_discounted_pagerank_reranked", "bm25_ReRank", "Doc2QueryT5_BM25", "Doc2QueryT5_BM25_ReRank", "BM 25 (Baseline)", "Sparse Cross Encoder", "RankZephyr"]
)

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100
0,bm25_and_discounted_pagerank,0.451294,0.555054,0.985294
1,bm25_and_discounted_pagerank_reranked,0.532779,0.653249,0.985294
2,bm25_ReRank,0.307790,0.477821,0.493176
3,Doc2QueryT5_BM25,0.287015,0.484460,0.565674
4,Doc2QueryT5_BM25_ReRank,0.245261,0.390143,0.487179
5,BM 25 (Baseline),0.374041,0.579877,0.601333
6,Sparse Cross Encoder,0.366460,0.612980,0.601333
7,RankZephyr,0.347070,0.568413,0.601333
